## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)
print(y_train.shape) 
print(x_test.shape) #(50000, 32, 32, 3)
print(y_test.shape) 

(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


In [4]:
np.mean(x_train,axis=(0))

array([[[130.71074, 136.05614, 132.5538 ],
        [130.14036, 135.44238, 131.85358],
        [131.05044, 136.24616, 132.58144],
        ...,
        [131.05902, 136.307  , 132.44078],
        [130.39308, 135.70848, 132.00036],
        [130.174  , 135.4736 , 131.89246]],

       [[130.0993 , 135.29484, 131.36412],
        [129.3446 , 134.45914, 130.4656 ],
        [130.2169 , 135.1767 , 131.10234],
        ...,
        [130.04254, 135.13862, 130.90018],
        [129.33038, 134.55898, 130.49918],
        [129.1568 , 134.38368, 130.45104]],

       [[129.72472, 134.64818, 130.2514 ],
        [128.71662, 133.49748, 129.01922],
        [129.47348, 134.05546, 129.47586],
        ...,
        [129.21066, 133.95824, 129.29398],
        [128.59094, 133.54152, 129.06528],
        [128.59134, 133.55772, 129.22356]],

       ...,

       [[126.55372, 125.9533 , 114.1421 ],
        [124.58844, 123.7335 , 111.77894],
        [123.92508, 122.8011 , 110.73544],
        ...,
        [124.24036, 123.00

In [15]:
np.mean(x_train,axis=(0,1))

array([[126.83261937, 128.18296813, 119.07407688],
       [124.75662562, 125.67388125, 116.54274875],
       [124.40423875, 124.74341   , 115.60854375],
       [124.1411475 , 123.91452313, 114.75189438],
       [124.10594875, 123.32259312, 114.11807375],
       [124.3007475 , 122.94016063, 113.7183175 ],
       [124.59202   , 122.663455  , 113.45950688],
       [124.89568313, 122.45004062, 113.27039312],
       [125.23976437, 122.28028125, 113.11450563],
       [125.58722687, 122.13966937, 113.00094938],
       [125.89378437, 122.04229688, 112.9426725 ],
       [126.06106875, 121.88908125, 112.872215  ],
       [126.14097437, 121.7128175 , 112.752225  ],
       [126.22666375, 121.58175375, 112.67661437],
       [126.22117938, 121.43404437, 112.57928   ],
       [126.16845688, 121.31369063, 112.48007   ],
       [126.17077187, 121.32533688, 112.44411062],
       [126.2099275 , 121.39754188, 112.49705687],
       [126.21394625, 121.53286687, 112.60722375],
       [126.162265  , 121.71260

In [16]:
np.mean(x_train,axis=(0,1,2))

array([125.30691805, 122.95039414, 113.86538318])

In [17]:
np.mean(x_train,axis=(0,1,2,3))

120.70756512369792

In [3]:


## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

In [4]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()

y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [19]:
print(y_train)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


In [5]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32,3,3,input_shape=(32,32,3),activation='relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())


#卷積組合
classifier.add(Convolution2D(64,3,3,input_shape=(32,32,3),activation='relu'))
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))


#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(64,activation='relu')) #output_dim=100,activation=relu

#輸出 最後一層是輸出層，也就是要輸出十種0~9的分類值，一般我們都會使用softmax作為分類模型輸出層的啟動函數。

classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=5)

Instructions for updating:
Colocations handled automatically by placer.


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(32, 32, 3..., activation="relu")`
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), input_shape=(32, 32, 3..., activation="relu")`
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Instructions for updating:
Use tf.cast instead.
Epoch 1/5
50000/50000 [==============================] - 865s 17ms/step - loss: 1.4405 - acc: 0.4937
Epoch 2/5
50000/50000 [==============================] - 746s 15ms/step - loss: 0.9899 - acc: 0.6504
Epoch 3/5
50000/50000 [==============================] - 728s 15ms/step - loss: 0.8166 - acc: 0.7137
Epoch 4/5
50000/50000 [==============================] - 767s 15ms/step - loss: 0.6844 - acc: 0.7606
Epoch 5/5
50000/50000 [==============================] - 765s 15ms/step - loss: 0.5617 - acc: 0.8027


## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [6]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[9.6485041e-02, 1.6416560e-06, 6.6288776e-06, 3.2779095e-01,
        2.5224859e-01, 9.0851836e-06, 2.2456932e-04, 2.4545029e-08,
        3.2321939e-01, 1.4088622e-05]], dtype=float32)